In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [5]:
%run -i ../../core.py

In [2]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [3]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.mean(dummy_data_tensor)



In [11]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<0.01*X.size()[1]*(self.w), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [12]:
# prover calculates settings, send to verifier

theory_output = torch.mean(data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        return (torch.abs(torch.sum(X)-X.size()[1]*(self.w))<0.01*X.size()[1]*(self.w), self.w)
prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

Theory_output:  tensor(49.9700)
==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[0,300],"logrows":14,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":606,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [13]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.4108660221099854 seconds
Theory output:  tensor(49.9700)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 50.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18373208119716085496, 9977853902793730609, 17093442942004944878, 1769661249493325049]]], 'proof': '0235e0538210ff9ecb77b4f549966dfd721687be42f2e41c9f96d64949153f0c20528203b6b32c92e651f5057cde9a47d2b8a06c8f40439d6a160244c86c214515d7a1dac880706f99942114edc3fd4442c7227722928bd0323b0a05438801bc2126bf6f2b79b6adbcf08ffcc0975119e7d9613f03e2ddded7b7edab279d2f950e1e2499653c00a73651e03456b8f6a8785fbad676a4f8d6486530b1c02eb69e231bca82d82af137bd314357aadc477f6eb790d8800c2e89c92c46ad0006dd6213847ce7792eb70370dcf63abee9181d8c5a7b8e79906a854586c89f4031db651b2fd31cc68c6ba6179fdd00cc04574633c5956300e02106e6dbc5a3c26a9c5c2a45dc936399d26f749a5fb0fe54cd9d2fd097e8a4d0f19178caa2e5d52e776e28508d9aaf63c95f2933bf349b442771e5a48b834b0d15cdcbe9434ef2aa2b250100bc631333bee1f240d9de2c07ea99238c4e615cf179c02f651bfee65e

In [14]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[10512373747352303962, 11798585516934984832, 13421675179368312123, 2200257403316998104], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [18373208119716085496, 9977853902793730609, 17093442942004944878, 1769661249493325049]]]
proof boolean:  1.0
proof result 1 : 50.0
verified
